In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [ ]:
# loading api_key
load_dotenv(dotenv_path = "./.env")
api_key = os.getenv("GROQ_API_KEY")

In [ ]:
# defining LLM
llm = ChatGroq(
    model = "gemma2-9b-it",
    temperature = 0.6,
    api_key = api_key
)

In [ ]:
# passing the prompt to LLM and getting reponse back
response = llm.invoke("I want to start a new Indian restaurant. Suggest a one fancy name, NO PERAMBLE.")
print(response.content)

Spice & Saga 



In [ ]:
# defining Prompt
name_prompt = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to start a new {cuisine} restaurant. Suggest a one fancy name, NO PERAMBLE."
)
name_prompt.format(cuisine = "Indian")

'I want to start a new Indian restaurant. Suggest a one fancy name, NO PERAMBLE.'

In [ ]:
# define chain - passing the prompt to LLM
name_chain = LLMChain(
    llm = llm,
    prompt = name_prompt
)

<ipython-input-7-95f58520ee68>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  name_chain = LLMChain(


In [ ]:
# running the chain
restaurant_name = name_chain.run("Indian")
print(restaurant_name)

<ipython-input-8-424a2825aa00>:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  restaurant_name = name_chain.run("Indian")


Spice Odyssey 




In [ ]:
# Will pass the response from name_chain to another chain and get the menu itesm
# This is called "SimpleSequentialChain"
# input --> name_chain --> name (output) --> menu_chain --> menu (output)

In [ ]:
# creating menu_chain
menu_prompt = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu item for {restaurant_name} restaurant. Return it in comma seperated values. NO PERAMBLE."
)

In [ ]:
# creating menu_chain
menu_chain = LLMChain(
    llm = llm,
    prompt = menu_prompt
)

In [ ]:
# Createing Simple Sequential Chain
chain = SimpleSequentialChain(
    chains = [name_chain, menu_chain]
)

In [ ]:
# getting response
response = chain.run("Indian")
print(response)

Tandoori Chicken, Butter Chicken, Rogan Josh, Chana Masala, Palak Paneer, Saag Aloo, Lamb Vindaloo, Chicken Tikka Masala, Vegetable Biryani, Garlic Naan, Onion Kulcha, Samosas, Pakoras, Mango Lassi, Gulab Jamun 



In [ ]:
# we didnt get restaurant name in the response from above. we only got menu
# for multiple output or multiple input we use SequentialChain

In [ ]:
# name chain
name_prompt = PromptTemplate(
    input_variables = ['cuisine'],
    template = "I want to start a new {cuisine} restaurant. Suggest a one fancy name, NO PERAMBLE."
)
name_chain = LLMChain(
    llm = llm,
    prompt = name_prompt,
    output_key = "restaurant_name"
)

# menu chain
menu_prompt = PromptTemplate(
    input_variables = ['restaurant_name'],
    template = "Suggest some menu item for {restaurant_name} restaurant. Return it in comma seperated values. NO PERAMBLE."
)
menu_chain = LLMChain(
    llm = llm,
    prompt = menu_prompt,
    output_key = "menu_items"
)

# Creating SequentialChain
chain = SequentialChain(
    chains = [name_chain, menu_chain],
    input_variables = ["cuisine"],
    output_variables = ["restaurant_name", "menu_items"]
)

In [ ]:
# running the chain
response = chain(["Indian"])
response

<ipython-input-16-49011916bf16>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain(["Indian"])


{'cuisine': ['Indian'],
 'restaurant_name': "Avani's Spice  \n",
 'menu_items': 'Tandoori Chicken, Butter Chicken, Rogan Josh, Palak Paneer, Vegetable Biryani, Saag Aloo, Naan, Samosas, Pakoras, Mango Lassi, Chai  \n'}

In [ ]:
response = chain({"cuisine": "Indian"})
response

{'cuisine': 'Indian',
 'restaurant_name': 'Spice & Silk \n',
 'menu_items': 'Crispy Tofu with Chili Garlic Sauce, Sichuan Mapo Tofu, Spiced Lamb Chops, Peking Duck Pancakes, Kung Pao Chicken, Black Pepper Shrimp, Spicy Szechuan Noodles, Vegetable Lo Mein, Jasmine Rice, Mango Sticky Rice  \n'}

In [ ]:
# LLM contains 2 things:
# 1. Knowledge (which can be limited to let say 2021)
# 2. Reasoning 

# If you ask ChatGPT - what is the best chepest flight from Chennai to Delhi, today?
# He will not know it - knowledge limited to 2021
# we will give pulgin and then he will use Reasoning and search in pulgin, and give you a answer
# LLM = Reasoning + Knowledge

In [ ]:
# ************
# * Agents
# ************
from langchain.agents import AgentType, initialize_agent, load_tools

# defining tool
tools = load_tools(
    ['wikipedia', 'llm-math'],
    llm = llm
)

# initializing agent
agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

# running the agent
agent.invoke("When was Elon Musk born? What is his current age?")

<ipython-input-19-efe53d3e5eac>:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: I can find Elon Musk's birthdate on Wikipedia and use a calculator to determine his current age. 
Action: wikipedia
Action Input: Elon Musk

KeyboardInterrupt: 

In [ ]:
# using SerpApi as tool now
# serpapi = google search api
tools = load_tools(
    tool_names = ['serpapi', 'llm-math'],
    llm = llm
)
agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)
agent.invoke("What was the GDP of India in 2022 plus 5?")



> Entering new AgentExecutor chain...
Thought: I need to find the GDP of India in 2022 and then add 5 to it.
Action: Search
Action Input: GDP of India 2022
Observation: ["India, officially the Republic of India, is a country in South Asia. It is the seventh-largest country by area; the most populous country from June 2023 onwards; and since its independence in 1947, the world's most populous democracy.", 'India type: Country in South Asia.', 'India kgmid: /m/03rk0.', 'Economy of India ; –6.4% of GDP (2022–23) · ₹27.163 trillion (US$320 billion); (2023–24) · ₹45.031 trillion (US$530 billion); (2023–24) · Donor: Increase $4.234 ...', 'Nominal (current) Gross Domestic Product (GDP) of India is $3,567,550,000,000 (USD) as of 2023. · Real GDP (constant, inflation adjusted) of India reached ...', 'GDP (current US$) - India. World Bank national accounts data, and OECD ... 2022. 2,898.23. Bolivia. 2023. 45,135.40. Bosnia and Herzegovina. 2023. 27,514.78.', 'India GDP for 2022 was 3.353 trill

{'input': 'What was the GDP of India in 2022 plus 5?',
 'output': '8.353 trillion US dollars'}

In [ ]:
# *************************
#* Memory
# *************************
# by default LLMs are state-less (non memory)
chain = LLMChain(llm = llm, prompt = name_prompt)

chain.run("Indian").strip()

chain.run("Mexicon").strip()

type(chain.memory)

# I want to remember the chat
# will attached the memory
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(
    llm = llm,
    prompt = name_prompt,
    memory = memory             # have to add the memory in the chain
)

chain.run("Indian").strip()
chain.run("Mexicon").strip()

chain.memory
print(chain.memory.buffer)

<ipython-input-21-64deb080e0a4>:17: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


Human: Indian
AI: Spice & Saga  

Human: Mexicon
AI: El Sol Dorado 



In [ ]:
# i want to create a converstaion chain and not use LLMChain
# by default we have a convertation chain
from langchain.chains import ConversationChain

convo = ConversationChain(
    llm = llm
)

print(convo.prompt)
print(convo.prompt.template)

convo.run("Who won the first cricket world cup?")
convo.run("what is 5+78?")
convo.run("In which country the match had been played?")

convo.memory
print(convo.memory.buffer)

<ipython-input-22-eee29d4365a9>:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  convo = ConversationChain(


input_variables=['history', 'input'] input_types={} partial_variables={} template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:'
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:
Human: Who won the first cricket world cup?
AI: The West Indies won the first Cricket World Cup in 1975!  They beat Australia in the final held at Lord's Cricket Ground in London.  

Did you know that Clive Lloyd was the captain of the winning team?  🏏🏆  

Human: what is 5+78?
AI: 5 + 78 = 83 

Human: In which country the match had been played

In [ ]:
# Disadvantage = 
# it will keep on accumulating
# when call LLM  - all log will go there - and LLM charge per token (1 word = 1 token approx.)
# this increase the cost
# to handle that we can set how many past messages it should use
from langchain.memory import ConversationBufferWindowMemory

# here we set k = 2 --> LLM will remember last 2 conversation only.
memory = ConversationBufferWindowMemory(k = 2)

convo = ConversationChain(
    llm = llm,
    memory = memory                 # explicitly adding memory here - previously we had not added it (by default it is there). Now we had given our memory.
)

convo.run("Who won the first cricket world cup?")
convo.run("what is 5+78?")
print(convo.memory.buffer)
convo.run("what is sin(45) + tan(45)?")
print(convo.memory.buffer)
convo.run("In which country the match had been played?")
print(convo.memory.buffer)

<ipython-input-23-65d33e2a4705>:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k = 2)


Human: Who won the first cricket world cup?
AI: West Indies won the first Cricket World Cup in 1975!  They beat Australia in the final held at Lord's Cricket Ground in London.  🏏🏆  




Human: what is 5+78?
AI: 5 + 78 = 83 

Human: what is 5+78?
AI: 5 + 78 = 83 

Human: what is sin(45) + tan(45)?
AI: sin(45) is √2 / 2 and tan(45) is 1.  So, sin(45) + tan(45) = √2 / 2 + 1 

Human: what is sin(45) + tan(45)?
AI: sin(45) is √2 / 2 and tan(45) is 1.  So, sin(45) + tan(45) = √2 / 2 + 1 

Human: In which country the match had been played?
AI: AI: I need more context to answer that question.  What match are you referring to? ⚽️ 🏀 🏈  Please tell me the teams or the sport! 


Let me know and I'll do my best to find the country where the match was played! 🌎 

